In [7]:
### This notebook contains code that filter sgRNAs that only edit synonymous mutations
### The main difference from previous version is it fetch the BE window sequences from annotation file directly
import pandas as pd 
from tqdm import tqdm
import pickle
from Bio import SeqIO
import re

from os.path import exists,join
from os import listdir,makedirs

import sys
sys.path.append('../')
from src import syn_sgFinder
from itertools import chain,product

In [2]:
%load_ext autoreload
%autoreload 2

In [8]:
out_path = '../../data/output/sg_Finder'
mrna_path = '../../data/MANE'
fpath = '../../data/output/sg_out/'

# Load all expressed transcripts
ess_genelist = pd.read_csv('../../data/output/K562_comm_ess.txt', header =None)
neutral_genelist = pd.read_csv('../../data/output/K562_neutral.txt', header =None)

list_exp_trsp = pickle.load(open('../../data/22Q2_gene_effect/expressed_transcripts.pkl','rb'))
# Parse mRNA sequence dictionary, for use when sequence aligned to intron-exon junctions
record_dict = SeqIO.to_dict(SeqIO.parse(join(mrna_path,'MANE.GRCh38.v1.0.refseq_rna.fna'), "fasta"))

fpath = '../../data/sg_out/'
list_gene = [i for i in listdir(fpath) if not i.startswith('.')]
list_gene = [i for i in list_gene if len(listdir(join(fpath,i))) > 5]
list_gene_ess = [i for i in list_gene if i in list(ess_genelist[0])]
list_gene_neutral = [i for i in list_gene if i in list(neutral_genelist[0])]

In [9]:
def run_sgFinder(input_genes, length, tag = 'ess', sg_p ='../../data/output/sg_out' ):
    out_path = '../../data/output/sg_Finder'
    
    df_synsg_abe = pd.DataFrame()
    df_synsg_cbe = pd.DataFrame()
    df_abeCT = pd.DataFrame()
    df_cbeCT = pd.DataFrame()
    
    dict_filter = {} # Record filtered sgRNA index
    for genes in tqdm(input_genes):
        
        if not exists(join(out_path,genes,tag)):
            makedirs(join(out_path,genes,tag))
            
        # Get gene transcript, sequence information
        sg_gene = syn_sgFinder.sgFinder_gene(genes,sg_path = '../../data/sg_out' )
        sg_gene.gene_cds_proc(record_dict,list_exp_trsp) # filter out non-expressed transcript

        # Process sgRNA for each transcripts
        df_synsg_abe_dt = pd.DataFrame()
        df_synsg_cbe_dt = pd.DataFrame()
        for idx, transcripts in list(product(sg_gene.sg_file.index,\
                                                               sg_gene.dict_exons.keys())):
            sg_trsp = syn_sgFinder.sgFinder_transcript(sg_gene.gene,idx,transcripts)
            check_point = sg_trsp.process_BE_window(sg_gene,dict_filter, window_length = length)
            if check_point:
                sg_trsp.all_pos_consq(sg_gene)

                df_synsg_abe = df_synsg_abe.append(sg_trsp.df_abe,ignore_index = True)
                df_synsg_cbe = df_synsg_cbe.append(sg_trsp.df_cbe,ignore_index = True)

                df_abeCT = df_abeCT.append(sg_trsp.df_abe_control,ignore_index = True)
                df_cbeCT = df_cbeCT.append(sg_trsp.df_cbe_control,ignore_index = True)

                df_synsg_abe_dt = df_synsg_abe_dt.append(sg_trsp.df_abe_ind,ignore_index = True)
                df_synsg_cbe_dt = df_synsg_cbe_dt.append(sg_trsp.df_cbe_ind,ignore_index = True)

        df_synsg_abe_dt.to_csv(join(out_path,genes,tag,f'df_abe_detail.csv'))
        df_synsg_cbe_dt.to_csv(join(out_path,genes,tag,f'df_cbe_detail.csv'))

    # Process and saving final result
    ABE_synsg = df_synsg_abe[df_synsg_abe['syn_or_not'].apply(lambda x: all(x)&any(x))]
    CBE_synsg = df_synsg_cbe[df_synsg_cbe['syn_or_not'].apply(lambda x: all(x)&any(x))]

    if not exists(join(out_path,tag)):
        makedirs(join(out_path,tag))
    ABE_synsg.to_csv(join(out_path,tag,'ABE_synsg.csv'))
    CBE_synsg.to_csv(join(out_path,tag,'CBE_synsg.csv'))

    df_abeCT.to_csv(join(out_path,tag,'ABE_CT.csv'))
    df_cbeCT.to_csv(join(out_path,tag,'CBE_CT.csv'))
    return(dict_filter)

In [10]:
filter_15 = run_sgFinder(list_gene_ess, length = 15, tag = 'ess_15' )

  0%|          | 0/122 [00:00<?, ?it/s]

Processing AARS1...Keep 1039/1167 low off-target sgRNAs...


  1%|          | 1/122 [00:15<30:56, 15.35s/it]

Processing ABCE1...Keep 44/486 low off-target sgRNAs...


  2%|▏         | 2/122 [00:15<13:22,  6.69s/it]

...

 98%|█████████▊| 120/122 [15:46<00:16,  8.00s/it]

Processing WEE1...Keep 261/414 low off-target sgRNAs...


 99%|█████████▉| 121/122 [15:51<00:07,  7.38s/it]

Processing YARS1...Keep 500/574 low off-target sgRNAs...
YARS1 NM_003680                       sequence length: 2443,                       index length:3117


100%|██████████| 122/122 [15:52<00:00,  7.81s/it]


In [11]:
filter_neutral = run_sgFinder(list_gene_neutral, length = 15, tag = 'neutral_15' )

  8%|▊         | 3/40 [00:00<00:01, 28.51it/s]

Processing ATP6V1G3...Keep 0/39 low off-target sgRNAs...
Processing BRDT...Keep 517/722 low off-target sgRNAs...
Processing C10orf53...Keep 77/82 low off-target sgRNAs...
Processing CER1...Keep 293/359 low off-target sgRNAs...
CER1 NM_005454                       sequence length: 1231,                       index length:1229
Processing CETN1...Keep 208/244 low off-target sgRNAs...


 25%|██▌       | 10/40 [00:07<00:20,  1.47it/s]

Processing CHAT...Keep 700/779 low off-target sgRNAs...
Processing CNPY1...Keep 58/71 low off-target sgRNAs...
Processing CST9...Keep 150/194 low off-target sgRNAs...
Processing CST9L...Keep 116/173 low off-target sgRNAs...
Processing DEFA6...Keep 105/123 low off-target sgRNAs...
Processing EGR4...Keep 693/803 low off-target sgRNAs...
Processing FCRL4...Keep 420/583 low off-target sgRNAs...


 35%|███▌      | 14/40 [00:07<00:10,  2.49it/s]

...

Processing RFPL4B...Keep 269/337 low off-target sgRNAs...


 85%|████████▌ | 34/40 [01:06<00:13,  2.18s/it]

Processing RNASE11...Keep 175/227 low off-target sgRNAs...
Processing RPTN...Keep 527/1002 low off-target sgRNAs...
Processing SLC6A18...Keep 721/852 low off-target sgRNAs...
Processing TEX55...Keep 605/678 low off-target sgRNAs...
Processing TMPRSS15...Keep 765/903 low off-target sgRNAs...


100%|██████████| 40/40 [01:17<00:00,  1.94s/it]

Processing TPD52L3...Keep 130/150 low off-target sgRNAs...
Processing VN1R2...Keep 278/473 low off-target sgRNAs...


In [12]:
### Save filtered dataframe dictionary
values_2 = []
for keys in filter_15:
    values_2.append(len(filter_15[keys]))
values_1 = []
for keys in filter_neutral:
    values_1.append(len(filter_neutral[keys]))
filter_ess = pd.DataFrame({'gene':filter_15.keys(),'n_filtered':values_2})
filter_neutral = pd.DataFrame({'gene':filter_neutral.keys(),'n_filtered':values_1})
filter_ess.to_csv(join(out_path,'ess_15','filter_gene.csv'))
filter_neutral.to_csv(join(out_path,'neutral_15','filter_gene.csv'))

In [260]:
# When gene is + and sg is +  .seq[sequence_idx:sequence_idx+11]
# When gene is + and sg is - .seq[sequence_idx+12:sequence_idx+23].reverse_complement()
# When gene is - (the sequence is -, ATG) and sg is - .seq[sequence_idx-22:sequence_idx-11]
# When gene is - and sg is + .seq[sequence_idx-10:sequence_idx+1].reverse_complement()

Seq('CGCTTTCGCGC')